In [3]:
import numpy as np
'''
Prim 算法, 构建最小生成树

Prim 算法基本思路: 
    设一个图 G=<V, E>.
    设最小生成树所包含的边的集合 MST. 最开始MST=空集.
    维护一个集合 V_A, 往 V_A 中不断增加结点.
    每次选出连接 V_A 和 V-V_A 中顶点的权值最小的边, 把这条边加入到 MST 中.
    并且把这条边在 V-V_A 中的顶点加到 V_A 中.
    这样重复|V|-1 次, 找出|V|-1 条符合上述迭代方法的边, 就构成了最小生成树.
'''

'\nPrim 算法, 构建最小生成树\n\nPrim 算法基本思路: \n    设一个图 G=<V, E>.\n    设最小生成树所包含的边的集合 MST. 最开始MST=空集.\n    维护一个集合 V_A, 往 V_A 中不断增加结点.\n    每次选出连接 V_A 和 V-V_A 中顶点的权值最小的边, 把这条边加入到 MST 中.\n    并且把这条边在 V-V_A 中的顶点加到 V_A 中.\n    这样重复|V|-1 次, 找出|V|-1 条符合上述迭代方法的边, 就构成了最小生成树.\n'

In [4]:
def build_adjM(vertexNum:int, edges:list[tuple[int, int, int]])->np.ndarray:
    vertex_id=[i for i in range(vertexNum)]
    adjM=np.full(shape=(vertexNum, vertexNum), fill_value=np.inf, dtype=np.float64)

    # 不用numpy库实现 adjM 创建: adjM=[[0 for _ in range(numCourses)] for _ in range(numCourses)]

    for (u, v, weight) in edges:
        adjM[u, v]=adjM[v, u]=weight # 无向有权图

    return adjM

In [5]:
def Prim1(adjM:np.ndarray)->tuple[set[tuple[int, int]], int]:
    '''
    Prim 算法实现一: 
    设两个数组 dist 和 pred, 其中:
    dist[u]=w 表示 V_A 中的结点到 V-V_A中的结点u 的最短边权值
    pred[u]=v 表示上述 dist[u]=w 取到的时候, V_A 中与 u 相连的结点.

    每次通过排序, 维护 dist, pred, 进而找出权值最小的, 横跨 V_A, V-V_A 的边.
    把这条边(v, u) (v 属于 V_A, u 属于 V-V_A)中的 u 加入到 V_A 中, 然后如此迭代.
    并且把这条边加入 MST.

    如此重复|V|-1 次, 可以得到 MST.

    复杂度估计:
        主要复杂度花在找最小权值的横跨 V_A, V-V_A 的边上了. 复杂度大致在 O(|V|^3)左右?
        (prim1 函数的 for 循环中嵌入一个 update, update 函数大概是 O(|V|^2))
    '''
    
    verNum=adjM.shape[0]

    selected=set()

    dist=np.full(shape=verNum, fill_value=np.inf, dtype=np.float64) 
    # tip1: dist[u]=w 表示V_A中的顶点到V-V_A中的顶点u的最小边权值
    # tip2: np.inf 不和 np.int16 兼容, 应当换成 dtype=np.float64

    pred=[-1 for _ in range(verNum)] #  dist[u]=w 对应的前驱结点

    minimum_spanning_tree=set()
    mst_total_weight=0

    selected.add(0)
    pred[0]=-1

    def update(): # 更新此时的 dist[]数组, 找到权重最小的轻边
        for u in range(verNum):
            if not u in selected:
                for v in selected:
                    if adjM[v, u]<dist[u]:
                        dist[u]=adjM[v, u]
                        pred[u]=v

    
    for _ in range(verNum-1):
        update()
        min_dist=np.inf
        u_key=0
        for u in range(verNum):
            if not u in selected:
                if dist[u]<min_dist:
                    min_dist=dist[u]
                    u_key=u
        minimum_spanning_tree.add((pred[u_key], u_key))
        mst_total_weight+=adjM[pred[u_key], u_key]
        selected.add(u_key)
        # print(selected)
    
    # print(adjM)
    # print(pred)
    # print(dist)

    return minimum_spanning_tree, mst_total_weight

In [6]:
def Prim1_refine(adjM:np.ndarray)->tuple[set[tuple[int, int]], int]:
    '''
    Prim 算法实现一_改进版: 
    设两个数组 dist 和 pred, 其中:
    dist[u]=w 表示 V_A 中的结点到 V-V_A中的结点u 的最短边权值
    pred[u]=v 表示上述 dist[u]=w 取到的时候, V_A 中与 u 相连的结点.

    维护 dist, pred, 进而找出权值最小的, 横跨 V_A, V-V_A 的边(这里不使用 update, 比 Prim1 更快一些)
    把这条边(v, u) (v 属于 V_A, u 属于 V-V_A)中的 u 加入到 V_A 中, 然后如此迭代.
    并且把这条边加入 MST.

    如此重复|V|-1 次, 可以得到 MST.

    复杂度估计:
        相比之前, 只有两层循环, 复杂度 O(|V|^2).

    返回：
        minimum_spanning_tree: MST 的边集合（元组形式，(v, u) 表示从 v 到 u 的边）
        mst_total_weight: MST 的总权重
    '''
    
    verNum=adjM.shape[0]

    selected=[0 for _ in range(verNum)] # 当前节点是否被加入到 V_A 中.
    # =1: 已加入; =0: 未加入

    dist=np.full(shape=verNum, fill_value=np.inf, dtype=np.float64) 
    pred=[-1 for _ in range(verNum)] 
    # tip1: dist[u]=w 表示V_A中的顶点到V-V_A中的顶点u的最小边权值
    # tip2: np.inf 不和 np.int16 兼容, 应当换成 dtype=np.float64
    # tip3: dist[u]=w 表示已选集合 V_A 到未选节点 u 的最短边权值为 w
    # tip4: pred[u]=v表示, 对应dist[u]=w时对应的边(v, u)

    minimum_spanning_tree=set()
    mst_total_weight=0 # MST 中所有边的权重之和

    # selected[0]=1
    pred[0]=-1
    
    for _ in range(verNum):
        min_Dist=np.inf
        u_key=0
        for u in range(verNum): # 选出此时应该添加到 V_A 中的顶点及其对应的轻边(pred[u_key], u_key)
            if selected[u]==0 and dist[u]<min_Dist:
                min_Dist=dist[u]
                u_key=u
        
        if u_key!=0: # 因为pred[0]=-1, 不把这个添加到 MST 中
            v=pred[u_key]
            minimum_spanning_tree.add((v, u_key))
            mst_total_weight+=adjM[v][u_key]

        for u in range(verNum): 
        # u_key 添加到V_A 后, 更新V_A 到 V-V_A 中各个点的最短距离--也即 dist 数组
        # 同时更新存储dist[u]=w 时对应的边(v, u), 其中 pred[u]=v(见 pred 数组定义如上)
        # 只需比较 adjM[u_key][u]和 dist[u]哪个大就行
            tmp=adjM[u_key][u]
            if  tmp!=np.inf and tmp <dist[u]: 
                dist[u]=tmp
                pred[u]=u_key
        selected[u_key]=1
        
    # print(adjM)
    # print(pred)
    # print(dist)

    return minimum_spanning_tree, mst_total_weight

更快的实现方法? 在 Prim1_refine 基础上

Prim1 函数中, 第一层 for 循环内部, 第一个 for循环每次需要dist[u]的最小值, 相当于每次要对一个列表/数组求最小值.

对于这样多次找最小, 堆更适合, 或者说, 优先队列.(堆是优先队列的实现形式之一)

以下通过 ```heapq.py``` 的堆相关操作, 简化这一流程.

Prim2 函数便是在如下基础上构建的.

这部分内容借助 python 内置的优先队列.

(python 内置的有限队列对于多线程操作是安全的, 而 heapq 则是不安全的)

关于 heap, heapy, PriorityQueue, 可以参考该分支下 ```Heap_and_PriorityQueue.py```

**总结: 堆的"用武之地":适合处理多次寻找最值的问题, 且每次寻找到一个当前最值后, 当前最值被使用而不参与之后轮次的最值寻找**

In [ ]:
import heapq
def Prim_PriQue(adjM:np.ndarray)->tuple[set[tuple[int, int]], int]:
    verNum=adjM.shape[0]

    selected=[0 for _ in range(verNum)]

    dist=np.full(shape=verNum, fill_value=np.inf, dtype=np.float64) 
    pred=[-1 for _ in range(verNum)] 
    # tip1: dist[u]=w 表示V_A中的顶点到V-V_A中的顶点u的最小边权值
    # tip2: np.inf 不和 np.int16 兼容, 应当换成 dtype=np.float64
    # tip3: dist[u]=w 表示已选集合 V_A 到未选节点 u 的最短边权值为 w
    # tip4:pred[u]=v表示, 对应dist[u]=w时对应的边(v, u)
        
    minimum_spanning_tree=set() # MST
    mst_total_weight=0 # MST所有边权重之和

    pq=[] # 最小堆

    dist[0]=0
    # selected[0]=1 # Prim_Prique()这里不能先给 selected[0]赋值为 1, 需要在下面的循环中赋值, 否则与 0 相连的那条 MST 中的边没法加到 MST 的权重计算中
    pred[0]=-1
    

    # for u in range(verNum):
    #     heapq.heappush(pq, (dist[u], u))
    heapq.heappush(pq, (dist[0], 0))
    # 需要同时将dist[u]和 u 打包成元组传给堆, 防止排序后下标和值对不上
    # 此时排序的依据时元组第一个元素, 也就是 dist.
    # heapq默认采用小顶堆, 正好对应对 dist[]升序.
    
    while pq:
        current_dist, u_key=heapq.heappop(pq)

        if selected[u_key] == 1 or current_dist>dist[u_key]: # 具体解释见下面
            continue

        if pred[u_key] != -1:
            minimum_spanning_tree.add((pred[u_key], u_key))
            mst_total_weight += adjM[pred[u_key]][u_key]

        selected[u_key]=1

        for u in range(verNum):
            tmp=adjM[u_key][u]
            if tmp<np.inf and selected[u]==0 and tmp<dist[u]:
                dist[u]=tmp
                pred[u]=u_key
                heapq.heappush(pq, (dist[u], u))

                '''
                不应该将更新后的 (dist[u], u)代替原来的 (dist[u], u)放入堆中么, 为什么这里直接把新的放进去了, 旧的也保留?

                heapq 是「静态堆」，无法修改已推入堆中的旧值（比如把 (5, v) 改成 (3, v)），所以只能「推新值、留旧值」
                于是这样, 堆中可能存储同一个节点 v 的多个不同距离值（即 (距离值, v) 元组中, v 重复出现，仅距离值不同
                旧值 (5, v) 不会影响最终结果，因为后续弹出它时，会通过 current_dist > dist[v] 判定为「过时无效值」(上面十行左右的 if 判断)
                (但是上面 dist[u]=tmp, 已经把 dist[u]中的值修改过来了)
                当(3, v)入堆后, 假设此时push 后调整堆(heapq.heappush()会在入堆后将堆调整成小顶堆), 经过若干次push/pop 操作后,(3, v)调整至堆顶.
                进入下一次循环, pop 出(3, v), 正常执行. 此时(5, v)仍在堆内. (这里想强调的是, 由于小顶堆的特性:父节点值>子节点值, 因此(5, v)一定在(3, v)之后被 pop 出来)
                由于 heapq. heappop()每执行一次后不仅会 pop 最小元素, 还会将堆重新调整成小顶堆, 可以知道, 经过若干次后, 
                必有(5, v)位于堆顶的时候. 假设此时(5, v)是在堆顶, 那么调用 heapq.heappop, (5, v)出堆.
                此时 current_dist=5, u_key=v, 但是 dist[u_key]=3. if对这个(5, v)的判断结果是 continue.
                那么也就没有继续进行操作, 而是在弹出(5, v)后, 直接再次进入 while 循环. (5, v)的存在并没有对堆排序的这个过程产生实质性影响.

                因此, 可以通过上面的代码 push/pop 操作流, 维护一个小顶堆, 高效的维护 V_A 到 V-V_A 的轻边值.

                基于这一个原则, for u in range(verNum): heapq.heappush(pq, (dist[u], u))可以替换为只 push((dist[0], 0))
                '''

    # print(pred)
    # print(dist)

    return minimum_spanning_tree, mst_total_weight 

![Test Pic 1](MST_test1.png)

In [8]:
if __name__=='__main__':
    # 测试例子(如上图)来自 PPT P61, 结点字符已经转换成了对应的 id 编号在结点旁边
    vertexNum=9
    edges=[(0, 1, 4), (0, 4, 8), (1, 4, 1), (1, 2, 8), (2, 3, 7),
           (3, 8, 9), (7, 8, 10), (6, 7, 2), (4, 6, 1), (4, 5, 7),
           (2, 5, 2), (5, 6, 4), (2, 7, 4), (3, 7, 14)]
    
    adjM=build_adjM(vertexNum, edges)
    
    print('by means of Prim1:')
    minimum_spanning_tree, mst_total_weight=Prim1(adjM)
    print('MST includes edges:', minimum_spanning_tree)
    print('MST weight:', mst_total_weight)


    print('\nby means of Prim1_refine:')
    minimum_spanning_tree, mst_total_weight=Prim1_refine(adjM)
    print('MST includes edges:', minimum_spanning_tree)
    print('MST weight:', mst_total_weight)


    print('\nby means of Prim_PriQue:')
    minimum_spanning_tree, mst_total_weight=Prim_PriQue(adjM)
    print('MST includes edges:', minimum_spanning_tree)
    print('MST weight:', mst_total_weight)

by means of Prim1:
MST includes edges: {(0, 1), (3, 8), (4, 6), (1, 4), (2, 3), (6, 7), (7, 2), (2, 5)}
MST weight: 30.0

by means of Prim1_refine:
MST includes edges: {(0, 1), (3, 8), (4, 6), (1, 4), (2, 3), (6, 7), (7, 2), (2, 5)}
MST weight: 30.0

by means of Prim_PriQue:
[(np.float64(4.0), 1), (np.float64(8.0), 4)]
[(np.float64(1.0), 4), (np.float64(8.0), 4), (np.float64(8.0), 2)]
[(np.float64(1.0), 6), (np.float64(7.0), 5), (np.float64(8.0), 2), (np.float64(8.0), 4)]
[(np.float64(2.0), 7), (np.float64(4.0), 5), (np.float64(8.0), 2), (np.float64(8.0), 4), (np.float64(7.0), 5)]
[(np.float64(4.0), 2), (np.float64(4.0), 5), (np.float64(8.0), 2), (np.float64(8.0), 4), (np.float64(7.0), 5), (np.float64(14.0), 3), (np.float64(10.0), 8)]
[(np.float64(2.0), 5), (np.float64(4.0), 5), (np.float64(7.0), 3), (np.float64(7.0), 5), (np.float64(10.0), 8), (np.float64(14.0), 3), (np.float64(8.0), 2), (np.float64(8.0), 4)]
[(np.float64(4.0), 5), (np.float64(7.0), 5), (np.float64(7.0), 3), (np.float

![Test Pic 2](MST_test2.png)

In [16]:
if __name__=='__main__':
    # 以上例子来源: 数据结构-第九课-P63.
    # 图中结点已编号在旁边
    # 最小生成树如图中红色的边所示
    
    vertexNum=6
    edges=[(0, 2, 19), (0, 3, 20), (2, 3, 22), (0, 1, 16), (1, 3, 11),
           (3, 4, 14), (2, 4, 18), (1, 4, 6), (1, 5, 5), (4, 5, 9)]
    
    adjM=build_adjM(vertexNum, edges)

    print('by means of Prim1:')
    minimum_spanning_tree, mst_total_weight=Prim1(adjM)
    print('MST includes edges:', minimum_spanning_tree)
    print('MST weight:', mst_total_weight)


    print('\nby means of Prim1_refine:')
    minimum_spanning_tree, mst_total_weight=Prim1_refine(adjM)
    print('MST includes edges:', minimum_spanning_tree)
    print('MST weight:', mst_total_weight)


    print('\nby means of Prim1_PriQue:')
    minimum_spanning_tree, mst_total_weight=Prim_PriQue(adjM)
    print('MST includes edges:', minimum_spanning_tree)
    print('MST weight:', mst_total_weight)

by means of Prim1:
MST includes edges: {(0, 1), (1, 5), (4, 2), (1, 4), (1, 3)}
MST weight: 56.0

by means of Prim1_refine:
MST includes edges: {(0, 1), (1, 3), (1, 4), (1, 5)}
MST weight: 38.0

by means of Prim1_PriQue:
[(np.float64(16.0), 1), (np.float64(20.0), 3), (np.float64(19.0), 2), (np.float64(inf), 3), (np.float64(inf), 4), (np.float64(inf), 2), (np.float64(inf), 1), (np.float64(inf), 5)]
[(np.float64(5.0), 5), (np.float64(6.0), 4), (np.float64(inf), 1), (np.float64(19.0), 2), (np.float64(11.0), 3), (np.float64(inf), 2), (np.float64(inf), 5), (np.float64(inf), 3), (np.float64(20.0), 3), (np.float64(inf), 4)]
[(np.float64(6.0), 4), (np.float64(11.0), 3), (np.float64(inf), 1), (np.float64(19.0), 2), (np.float64(inf), 4), (np.float64(inf), 2), (np.float64(inf), 5), (np.float64(inf), 3), (np.float64(20.0), 3)]
[(np.float64(11.0), 3), (np.float64(18.0), 2), (np.float64(inf), 1), (np.float64(19.0), 2), (np.float64(inf), 4), (np.float64(inf), 2), (np.float64(inf), 5), (np.float64(inf